# Домашнее задание 8

### Задание 1 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [1]:
import pandas as pd

In [4]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [32]:
ratings_grouped = ratings.groupby('userId').count()
ratings_grouped = ratings_grouped[ratings_grouped.rating > 100].reset_index()
user_ids = ratings_grouped['userId'].tolist()
ratings = ratings[ ratings['userId'].isin(user_ids) ]
ratings_min_max = ratings.groupby('userId').agg([min, max])
ratings_min_max['mean'] = ratings_min_max['timestamp']['max'] - ratings_min_max['timestamp']['min']
mean_live = ratings_min_max['mean'].mean()/ 24 / 3600
print('среднее время жизни пользователей, которые поставили больше 100 оценок {:.2f} дней'.format(mean_live) )

среднее время жизни пользователей, которые поставли больше 100 оценок 463.89 дней


### Задание 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [33]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd


In [34]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [35]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [36]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [42]:
revenue_without_adress = rzd.merge(auto, on ='client_id', how = 'outer').merge(air,on ='client_id', how = 'outer')
revenue_without_adress

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [44]:
revenue_wit_adress = (client_base.merge(rzd,on ='client_id', how = 'outer').merge(auto, on ='client_id', how = 'outer').
                      merge(air,on ='client_id', how = 'outer'))
revenue_wit_adress

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


### Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

#### Решение

### Если я правильно поняла каким должен быть ответ:
#### Если есть  офф-лайн точка продаж
1. Доработала бы данные с последовательностью координат.
Ввела бы новые столбцы.
Один - "визит" -  в нем в соответствии с условием, если координаты пользователя совпадают с координатами точки, столбец заполняется 1 или 0
Второй - "находился рядом"  в нем в соответвии с условием, если координаты пользователя удалены от точки магазины на определенное расстояние (задается опционально) столбец также заполняется 1 или 0
2. Получила бы 3 новые таблицы
первая - результат группировки по столбцу "визит" (считали количество посещений)
вторая - результат группировки по столбцу "находился рядом" (считали сколько раз был рядом)
третья - уникальные значения user_id 
3. Соединила бы все три новые таблицы в одну - "геолокация" через merge ('outer')
4. Соединила бы таблицу визитов и покупок с  "геолокация" через merge ('outer')
итог  - у нас есть таблица, по которой можно посмотреть данные  как часто пользователь проходит мимо нашей точки, сколько раз все таки зашел и сколько раз купил. Можно считать конверсию, определять как использовать рекламу и т.д.

#### Если инет магазин
1. на основе данных по местоположению пользователя создать сводную таблицу, где в строках user_id, а в столбцах гео данные. В ячейках подсчитывается количество совпадений (пользователь-гео точка)
2. таблицу визитов и покупок также группируем по user_id, считаем общее количество покупок, визитов
3. объеденяем таблицы через join и получаем данные user_id, общее количество визитов и расшифровку с какой гео точки чаще всего заходили